In [42]:
import pandas as pd

imdb_df = pd.read_csv("./MovieData/movie_metadata 2.csv")

In [43]:
imdb_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [44]:
imdb_df.keys()

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [140]:
imdb_df['title_year'].max()

2016.0

In [45]:
df_movie_imdb_link = imdb_df['movie_imdb_link']

In [46]:
movi_list = df_movie_imdb_link.to_list()

In [47]:
df_movie_imdb_link

0       http://www.imdb.com/title/tt0499549/?ref_=fn_t...
1       http://www.imdb.com/title/tt0449088/?ref_=fn_t...
2       http://www.imdb.com/title/tt2379713/?ref_=fn_t...
3       http://www.imdb.com/title/tt1345836/?ref_=fn_t...
4       http://www.imdb.com/title/tt5289954/?ref_=fn_t...
                              ...                        
5038    http://www.imdb.com/title/tt3000844/?ref_=fn_t...
5039    http://www.imdb.com/title/tt2071645/?ref_=fn_t...
5040    http://www.imdb.com/title/tt2107644/?ref_=fn_t...
5041    http://www.imdb.com/title/tt2070597/?ref_=fn_t...
5042    http://www.imdb.com/title/tt0378407/?ref_=fn_t...
Name: movie_imdb_link, Length: 5043, dtype: object

In [48]:
box_office = pd.read_csv("./MovieData/boxofficemojo_dataset/Mojo_budget_data.csv")

In [49]:
box_office.head()

,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
0,tt0118589,Glitter,2001,Vondie Curtis-Hall,Cheryl L. West,Laurence Mark,Terence Blanchard,Geoffrey Simpson,Mariah Carey,Eric Benét,...,4274407,997259,5271666,PG-13,1 hr 44 min,Drama,Music,Romance,NaN,https://www.boxofficemojo.com/title/tt0118589/...
1,tt0120630,Chicken Run,2000,Peter Lord,Peter Lord,Peter Lord,Harry Gregson-Williams,Simon Jacobs,Mel Gibson,Julia Sawalha,...,106834564,118000000,224834564,NaN,1 hr 24 min,Adventure,Animation,Comedy,Drama,https://www.boxofficemojo.com/title/tt0120630/...
2,tt0120667,Fantastic Four,2005,Tim Story,Mark Frost,Avi Arad,John Ottman,Oliver Wood,Ioan Gruffudd,Michael Chiklis,...,154696080,178839854,333535934,PG-13,1 hr 46 min,Action,Adventure,Family,Fantasy,https://www.boxofficemojo.com/title/tt0120667/...
3,tt0120679,Frida,2002,Julie Taymor,Hayden Herrera,Lindsay Flickinger,Elliot Goldenthal,Rodrigo Prieto,Salma Hayek,Alfred Molina,...,25885000,30413474,56298474,R,2 hr 3 min,Biography,Drama,Romance,NaN,https://www.boxofficemojo.com/title/tt0120679/...
4,tt0120681,From Hell,2001,Albert Hughes,Alan Moore,Jane Hamsher,Trevor Jones,Peter Deming,Johnny Depp,Heather Graham,...,31602566,42955549,74558115,R,2 hr 2 min,Horror,Mystery,Thriller,NaN,https://www.boxofficemojo.com/title/tt0120681/...


In [50]:
box_office.keys()

Index(['movie_id', 'movie_title', 'movie_year', 'director', 'writer',
       'producer', 'composer', 'cinematographer', 'main_actor_1',
       'main_actor_2', 'main_actor_3', 'main_actor_4', 'budget', 'domestic',
       'international', 'worldwide', 'mpaa', 'run_time', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'link'],
      dtype='object')

In [51]:
imdb_df["movie_id"] = imdb_df.movie_imdb_link.str[26:35]


In [52]:
imdb_df["movie_id"].head()
        

0    tt0499549
1    tt0449088
2    tt2379713
3    tt1345836
4    tt5289954
Name: movie_id, dtype: object

In [193]:
merged_df = pd.merge(imdb_df, box_office, on='movie_id')

In [194]:
merged_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,domestic,international,worldwide,mpaa,run_time,genre_1,genre_2,genre_3,genre_4,link
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,760507625,2029931375,2790439000,PG-13,2 hr 42 min,Action,Adventure,Fantasy,Sci-Fi,https://www.boxofficemojo.com/title/tt0499549/...
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,309420425,651576067,960996492,PG-13,2 hr 49 min,Action,Adventure,Fantasy,NaN,https://www.boxofficemojo.com/title/tt0449088/...
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,200074609,680600000,880674609,PG-13,2 hr 28 min,Action,Adventure,Thriller,NaN,https://www.boxofficemojo.com/title/tt2379713/...
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,448139099,632902188,1081041287,PG-13,2 hr 44 min,Action,Adventure,NaN,NaN,https://www.boxofficemojo.com/title/tt1345836/...
4,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,73078100,211061000,284139100,PG-13,2 hr 12 min,Action,Adventure,Sci-Fi,NaN,https://www.boxofficemojo.com/title/tt0401729/...


In [195]:
len(merged_df)

1986

In [196]:
merged_df['movie_year'].max()

2016

In [197]:
len(imdb_df)

5043

In [198]:
mask = merged_df.country.str.contains('USA')
# mask = ~merged_df['country'].str.contains('USA')
# print(merged_df[mask].country)
# use the boolean mask to drop the rows that don't contain 'hello'

merged_df = merged_df.drop(index=merged_df[~mask].index)

In [199]:
len(merged_df)

1675

In [200]:
merged_df.keys()

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title_x', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget_x', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'movie_id',
       'movie_title_y', 'movie_year', 'director', 'writer', 'producer',
       'composer', 'cinematographer', 'main_actor_1', 'main_actor_2',
       'main_actor_3', 'main_actor_4', 'budget_y', 'domestic', 'international',
       'worldwide', 'mpaa', 'run_time', 'genre_1', 'genre_2', 'genre_3',
       'genre_4', 'link'],
      dtype='object')

In [201]:
# removing duplicates and urelated attributes
merged_df = merged_df.drop(['run_time', 'budget_y', 'director_name', 'genres', 'link', 'movie_imdb_link', 'num_voted_users', 'movie_title_y', 'num_critic_for_reviews', 'content_rating', 'country'], axis=1)

In [202]:
merged_df = merged_df.drop('title_year', axis=1)

In [203]:
# make sure all the movies have at least 1 genre
merged_df = merged_df[merged_df['genre_1'].notna()]

In [204]:
len(merged_df)

1675

In [205]:
merged_df['movie_year'].max()

2016

In [206]:
merged_df['movie_year'].min()

2000

In [207]:
merged_df[['budget_x', 'movie_year']].head()

,budget_x,movie_year
0,237000000.0,2009
1,300000000.0,2007
3,250000000.0,2012
4,263700000.0,2012
5,258000000.0,2007


In [208]:
# The dollar had an average inflation rate of 2.10% per year between 2000 and 2016. Source: https://www.officialdata.org/2000-dollars-in-2016?amount=3900#:~:text=The%20dollar%20had%20an%20average,Labor%20Statistics%20consumer%20price%20index.
inflation = 1.021
for index, row in merged_df.iterrows():
    power = row['movie_year'] - 2000
    merged_df.at[index, 'budget_x'] = row['budget_x'] * (inflation ** power)
    merged_df.at[index, 'worldwide'] = row['worldwide'] * (inflation ** power)
    merged_df.at[index, 'domestic']= row['domestic'] * (inflation ** power)

In [209]:
merged_df[['budget_x', 'movie_year']].head()

,budget_x,movie_year
0,2.857459e+08,2009
1,3.469776e+08,2007
3,3.208108e+08,2012
4,3.383912e+08,2012
5,2.984007e+08,2007
